# ELMo
ELMo learns contextualized word vectors by running the text through a deep recurrent network.  
ELMo is actually an algorithm for unsupervised learning and does not make any use of the labels we have for our text classification task. The authors do show that contextualized word vectors obtained using ELMo increase text classification performance in a large array of tasks. Let's see if we see a significant gain in our case!  

There are good examples of using ELMo in both [the AllenNLP github repo](https://github.com/allenai/allennlp/blob/master/tutorials/how_to/elmo.md) and [this AnalyticsVidhya post](https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/?utm_source=blog&utm_medium=top-pretrained-models-nlp-article). In this guide we'll use the python package [Flair](https://github.com/zalandoresearch/flair) to get ELMo embeddings.

## Contextual word embeddings with ELMo in Flair
In Flair, you init a `Sentence` object given the tokens seperated by spaces.  
Sentence has a few useful attributes and methods

In [ ]:
!pip install allennlp
!pip install flair

In [5]:
from flair.embeddings import Sentence
sentence = Sentence('The grass is green .')

We also init a class of the desired embedding method. The `embed` method of this class gets a Sentence and adds to its tokens the relevant embedding.

In [13]:
from flair.embeddings import ELMoEmbeddings

# init embedding
elmo_embedding = ELMoEmbeddings()

elmo_embedding.embed(sentence)
for token in sentence:
    print(token)
    print(token.embedding.shape)
    print(token.embedding)

Token: 1 The
torch.Size([3072])
tensor([-0.3288,  0.2022, -0.5940,  ..., -1.2823,  0.3074,  0.2155])
Token: 2 grass
torch.Size([3072])
tensor([ 0.2539, -0.2363,  0.5263,  ..., -0.7016,  0.8834,  1.4157])
Token: 3 is
torch.Size([3072])
tensor([ 0.1915,  0.2300, -0.2894,  ..., -0.3680,  1.9113,  1.4519])
Token: 4 green
torch.Size([3072])
tensor([ 0.1779,  0.1309, -0.1041,  ..., -0.1034,  1.6204,  0.3276])
Token: 5 .
torch.Size([3072])
tensor([-0.8872, -0.2004, -1.0601,  ..., -0.0084, -0.0877,  0.0609])


**Try it yourself:** Now, compare the embeddings obtained using ELMo for the same word in different contexts. Are they equal or different?

### Word sense disambiguation using ELMo
**Try it yourself:** Let's also try to see how ELMo handles word sense disambiguation. Below are 6 sentences with 2 different meanings of the word `bank`. Try to see if ELMo vectors indeed separate the two meanings.

In [ ]:
sentences = [
    "I was walking along the river bank",
    "I saw a toad near the east bank of the river",
    "We had a nice picnic by the bank",
    "I need to deposit money from the bank",
    "The bank branch is closed",
    "He started working at the bank"
]

### Classify documents using the average of contextual word vectors
**Try it yourself:** *Optional:* In previous sections we've built a classifier using the average of non-contextual word vectors. Now, try to use contextual word embeddings on our dataset. Use the average of these vectors and apply a classifier on it to obtain the predictions. Is the performance better than for non-contextual word vectors?

### Sentence embedding using ELMo
We've used Flair to get embeddings for each word in the sentence. However, for text classification of the entire document, we need a way to integrate all these vectors into a single document embedding. There are several methods for that, and those interested would find this article useful - https://towardsdatascience.com/document-embedding-techniques-fed3e7a6a25d

The most basic element is averaging the word embedding into a single document embedding. In FLAIR, we do this using a DocumentPooling.

In [14]:
from flair.embeddings import DocumentPoolEmbeddings
document_embeddings = DocumentPoolEmbeddings([elmo_embedding], pooling='mean')

document_embeddings.embed(sentence)

# now check out the embedded sentence.
print(sentence.get_embedding().shape)
print(sentence.get_embedding())

torch.Size([3072])
tensor([-0.1185,  0.0253, -0.3043,  ..., -0.4927,  0.9270,  0.6944],
       grad_fn=<CatBackward>)


In [ ]:
document_embeddings

Alternatively, we can use an RNN that runs over the word embeddings. We will use the last hidden state as the document embedding. In this case it is very helpful to train the model using the true labels of our task, so that the RNN is optimized for our own data and task:

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings
glove_embedding = WordEmbeddings('glove')
document_embeddings = DocumentRNNEmbeddings([glove_embedding], rnn_type='LSTM')

In [ ]:
# create an example sentence
sentence = Sentence('The grass is green . And the sky is blue .')

# embed the sentence with our document embedding
document_embeddings.embed(sentence)

# now check out the embedded sentence.
print(sentence.get_embedding())

Note that while `DocumentPoolEmbeddings` are immediately meaningful, `DocumentRNNEmbeddings` need to be tuned on the downstream task. This happens automatically in Flair if you train a new model with these embeddings. You can find an example of training a text classification model [here](/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md#training-a-text-classification-model). Once the model is trained, you can access the tuned DocumentRNNEmbeddings object directly from the classifier object and use it to embed sentences.

`DocumentRNNEmbeddings` have a number of hyper-parameters that can be tuned to improve learning:

In [ ]:
:param hidden_size: the number of hidden states in the rnn.
:param rnn_layers: the number of layers for the rnn.
:param reproject_words: boolean value, indicating whether to reproject the token embeddings in a separate linear
layer before putting them into the rnn or not.
:param reproject_words_dimension: output dimension of reprojecting token embeddings. If None the same output
dimension as before will be taken.
:param bidirectional: boolean value, indicating whether to use a bidirectional rnn or not.
:param dropout: the dropout value to be used.
:param word_dropout: the word dropout value to be used, if 0.0 word dropout is not used.
:param locked_dropout: the locked dropout value to be used, if 0.0 locked dropout is not used.
:param rnn_type: one of 'RNN' or 'LSTM'

### Loading dataset
The simplest way to load our data in Flair is using a CSV file. You can learn about other method in [the documentation](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md).

In [ ]:
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = '/path/to/data'

# column format indicating which columns hold the text and label(s)
column_name_map = {4: "text", 1: "label_topic", 2: "label_subtopic"}

# load corpus containing training, test and dev data and if CSV has a header, you can skip it
corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=True,
                                         delimiter='\t',    # tab-separated files
) 

As previously mentioned, to create a `Corpus` for a text classification task, you need to have three files (train, dev, and test) in the 
above format located in one folder. This data folder structure could, for example, look like this for the IMDB task:
```text
/resources/tasks/imdb/train.txt
/resources/tasks/imdb/dev.txt
/resources/tasks/imdb/test.txt
```
Now create a `ClassificationCorpus` by pointing to this folder (`/resources/tasks/imdb`). 
Thereby, each line in a file is converted to a `Sentence` object annotated with the labels.

Attention: A text in a line can have multiple sentences. Thus, a `Sentence` object can actually consist of multiple
sentences.


In [ ]:
# load corpus containing training, test and dev data
corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.txt',
                                      dev_file='dev.txt',
                                      train_file='train.txt')

### Training our own model
We're going to use an RNN to run through the contextual word embeddings we got from ELMo. We will use the hidden state at the end of the document as an embedding for the entire document. We will train the RNN on our labeled dataset, so that the final hidden state carries the most relevant information for our custom classification task.  

For more information on training your own model using Flair, see [this tutorial](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_7_TRAINING_A_MODEL.md).

In [15]:
#TODO: Change code below to fit our own dataset

In [ ]:
rom flair.data import Corpus
from flair.datasets import TREC_6
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


# 1. get the corpus
corpus: Corpus = TREC_6()

# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   # FlairEmbeddings('news-forward'),
                   # FlairEmbeddings('news-backward'),
                   ]

# 4. initialize document embedding by passing list of word embeddings
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

# 7. start the training
trainer.train('resources/taggers/ag_news',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

# 8. plot weight traces (optional)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_weights('resources/taggers/ag_news/weights.txt')

## Contextual Word Vectors with BERT
We will later use BERT, a state-of-the-art transformer model that was trained on a very large corpus and can be fine-tuned for our own custom task. The Flair package can also be used to derive contextual word embeddings using BERT and its successors. We will use a different package for BERT, to provide you with sample code for using it, and for adaptation of the weights of the BERT model itself for our own task.

Below, you can try to use BERT, Roberta, XLNet or other models provided in Flair for contextual word embeddings.  
Flair also provides a simple way to stack vectors from different methods.

In [4]:
from flair.embeddings import FlairEmbeddings, BertEmbeddings

# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# init multilingual BERT
bert_embedding = BertEmbeddings('bert-base-multilingual-cased')

2019-10-17 08:38:42,789 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/lm-multi-forward-v0.1.pt not found in cache, downloading to C:\Users\Omri\AppData\Local\Temp\tmppppabpwb


100%|██████████████████████████████████████████████████████████████████| 73034300/73034300 [00:08<00:00, 8530759.25B/s]


2019-10-17 08:38:51,700 copying C:\Users\Omri\AppData\Local\Temp\tmppppabpwb to cache at C:\Users\Omri\.flair\embeddings\lm-multi-forward-v0.1.pt
2019-10-17 08:38:51,779 removing temp file C:\Users\Omri\AppData\Local\Temp\tmppppabpwb
2019-10-17 08:38:52,276 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/lm-multi-backward-v0.1.pt not found in cache, downloading to C:\Users\Omri\AppData\Local\Temp\tmptvnijhdp


100%|██████████████████████████████████████████████████████████████████| 73034304/73034304 [00:12<00:00, 5879942.68B/s]


2019-10-17 08:39:05,094 copying C:\Users\Omri\AppData\Local\Temp\tmptvnijhdp to cache at C:\Users\Omri\.flair\embeddings\lm-multi-backward-v0.1.pt
2019-10-17 08:39:05,172 removing temp file C:\Users\Omri\AppData\Local\Temp\tmptvnijhdp


100%|███████████████████████████████████████████████████████████████| 714314041/714314041 [01:08<00:00, 10488502.38B/s]


In [8]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
from flair.embeddings import StackedEmbeddings

# now create the StackedEmbedding object that combines all embeddings
stacked_embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, bert_embedding])

In [10]:
sentence = Sentence('The grass is green .')

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-1.4812e-07,  4.5007e-08,  6.0273e-07,  ...,  3.8287e-01,
         4.7210e-01,  2.9850e-01])
Token: 2 grass
tensor([ 1.6254e-04,  1.8764e-07, -7.9038e-09,  ...,  8.5283e-01,
        -5.0726e-02,  3.4476e-01])
Token: 3 is
tensor([-2.4521e-04,  3.4869e-07,  5.5841e-06,  ..., -1.8283e-01,
         7.1532e-01,  5.0841e-03])
Token: 4 green
tensor([8.3005e-05, 4.7261e-08, 5.7315e-07,  ..., 1.0157e+00, 7.5358e-01,
        1.1230e-01])
Token: 5 .
tensor([-8.3244e-07,  1.6451e-07, -1.7201e-08,  ..., -6.0930e-01,
         9.0591e-01,  1.7857e-01])


**Try it yourself:** Train a classifier using stacked embeddings of different models. Do you see an increase in performance?